In [1]:
import os
import glob
import codecs
import spacy
import json
import pandas as pd
import numpy as np
import networkx as nx

import nltk
from nltk.tokenize import sent_tokenize
import spacy
from spacy import displacy
from collections import Counter

from pathlib import Path
from pyspark import SparkContext
from afinn import Afinn
from sklearn.feature_extraction.text import CountVectorizer
import en_core_web_sm
import re

In [2]:
sent_file = open(r'C:\Users\NYashch\nyashch-projects\all_sentences.txt','r',encoding = 'utf-8')
all_sentences = [line.strip() for line in sent_file]
sent_file.close()
tolkien_text = ''.join(all_sentences)

In [5]:
tolkien_text=tolkien_text.lower()

Spacy

In [17]:
#add common words and races
cm_file = open(r'C:\Users\NYashch\nyashch-projects\Diploma\common_words.txt', 'r')
cm_lines = [line.strip() for line in cm_file]
cm_file.close()
common_words = []
for word in cm_lines:
    all_cm = word.split(',')
    for cm in all_cm:
        cm = cm.replace('"','')
        common_words.append(cm)
races = ['trolls','troll', 'werewolves','werewolf', 'half_elven','elf', 'ents','ent', 'eagles','eagle','dragons','dragon', 'barlogs','barlog','ainur','hobbit','hobbits','dwarves','dwarf', 'elves', 'maiar', 'orcs', 'orc', 'valar','valars']

In [9]:
def name_entity_recognition(sentence,names_list):
    '''
    A function to retrieve name entities in a sentence.
    :param sentence: the sentence to retrieve names from.
    :return: a name entity list of the sentence.
    '''

    doc = nlp(sentence)
    # retrieve person and organization's name from the sentence
    name_entity = [x for x in doc.ents if x.label_ in ['PERSON']]
    # convert all names to lowercase and remove 's in names
    name_entity = [str(x).lower().replace("'s","") for x in name_entity]
    # split names into single words ('Harry Potter' -> ['Harry', 'Potter'])
    name_entity = [x.split(' ') for x in name_entity]
    # flatten the name list
    name_entity = flatten(name_entity)
    # remove name words that are less than 3 letters to raise recognition accuracy
    name_entity = [x for x in name_entity if len(x) >= 3]
    # remove name words that are in the set of 4000 common words
    name_entity = [x for x in name_entity if x not in common_words]
    # remove name words that are in the set of races
    name_entity = [x for x in name_entity if x not in races]
    #print(name_entity)
    for name in name_entity:
        names_list.append(name)
    return name_entity
def flatten(input_list):
    '''
    A function to flatten complex list.
    :param input_list: The list to be flatten
    :return: the flattened list.
    '''

    flat_list = []
    for i in input_list:
        if type(i) == list:
            flat_list += flatten(i)
        else:
            flat_list += [i]

    return flat_list
nlp = en_core_web_sm.load()

In [18]:
import time
start_time = time.time()
spacy_ner = []
for sent in all_sentences:
    name_entity_recognition(sent,spacy_ner)
print("--- %s seconds ---" % (time.time() - start_time))


--- 914.1199584007263 seconds ---


NLTK

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [20]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [21]:
nltk_iobs = []
start_time = time.time()
for sent in all_sentences:
    iob = preprocess(sent)
    for i in iob:
        if list(i)[1] =='NNP':
            nltk_iobs.append(list(i)[0])
            #print((list(i)[0]))
print("--- %s seconds ---" % (time.time() - start_time))

--- 238.5028657913208 seconds ---


Golden standart

In [22]:
my_hobbits = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_hobbits.txt','r',encoding = 'utf-8')
hobbits = []
for elem in my_hobbits.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        hobbits.append(elem.lower().split()[0])

d_hobbits = dict.fromkeys(hobbits, 'hobbit')

my_dwarves = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_dwarves.txt','r',encoding = 'utf-8')
dwarves = []

for elem in my_dwarves.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        dwarves.append(elem.lower().split()[0])
d_dwarves = dict.fromkeys(dwarves, 'dwarf')
my_elves = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_elves.txt','r',encoding = 'utf-8')
elves = []
for elem in my_elves .readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        elves.append(elem.lower().split()[0])
d_elves = dict.fromkeys(elves, 'elf')
my_maiar = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_maiar.txt','r',encoding = 'utf-8')
maiar = []
for elem in my_maiar.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        maiar.append(elem.lower().split()[0])
d_maiar = dict.fromkeys(maiar, 'maiar')
my_orcs = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_orcs.txt','r',encoding = 'utf-8')
orcs = []
for elem in my_orcs.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        orcs.append(elem.lower().split()[0])
d_orcs = dict.fromkeys(orcs, 'orc')
my_men = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_men.txt','r',encoding = 'utf-8')
men = []
for elem in my_men.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        men.append(elem.lower().split()[0])
d_men = dict.fromkeys(men, 'men')
my_valars = open(r'C:\Users\NYashch\nyashch-projects\Diploma\my_valars.txt','r',encoding = 'utf-8')
valars = []
for elem in my_valars.readlines():
    if len(elem) > 4:
        elem = elem.replace("\n","")
        elem = elem.replace("\r","")
        valars.append(elem.lower())
d_valars = dict.fromkeys(valars, 'valar')
ainur = ["ainur",'aluin','danuin','fanuin','morgoth','ranuin']
d_ainur = dict.fromkeys(ainur, 'ainur')
barlogs = ['gothmog','lungorthin','osombauko']
d_barlogs = dict.fromkeys(barlogs, 'barlog')
dragons = ['ancalagon','chrysophylax','glaurung','gostir','scatha','smaug','urulókë']
d_dragons = dict.fromkeys(dragons, 'dragon')
eagles = ['gwaihir','landroval','meneldor','thornhoth','thorondor','windlord']
d_eagles = dict.fromkeys(eagles, 'eagle')
ents = ['beechbone','entings','entmaidens','entmoot','entwives','fimbrethil','leaflock','onodrim','quickbeam','skinbark','treebeard']
d_ents = dict.fromkeys(ents, 'ent')
half_elven = ['ardamir eärendil','arwen','dior','eärendil','elladan','elrohir','elrond','elros','elurín','elwing','undómiel']
d_half_elven = dict.fromkeys(half_elven, 'half_elf')
werewolves = ['carcharoth','draugluin','thû']
d_werewolves = dict.fromkeys(werewolves, 'werewolf')
trolls = ['bert','olog-hai','tom','william']
d_trolls = dict.fromkeys(trolls, 'troll')
all_names = hobbits + dwarves +elves+maiar+orcs+men+valars+ainur+barlogs+dragons+eagles+ents+half_elven+werewolves+trolls
names = list(set(all_names))
names.append('sam')

Precision and Recall

In [34]:
nltk_iobs = set(nltk_iobs)
#with NLTK
tp_nltk = 0
fp_nltk = 0
for iob in nltk_iobs:
    if iob.lower() in names:
        tp_nltk = tp_nltk +1
    else:
        fp_nltk = fp_nltk+ 1
fn_nltk = len(names) - tp_nltk
print('NLTK precision score is',tp_nltk / (tp_nltk + fp_nltk))
print('NLTK recall score is',tp_nltk / (tp_nltk + fn_nltk))
#with Spacy
spacy_ner = set(spacy_ner)
tp_spacy = 0
fp_spacy = 0
for ner in spacy_ner:
    if ner.lower() in names:
        tp_spacy = tp_spacy +1
    else:
        fp_spacy = fp_spacy+ 1
fn_spacy = len(names) - tp_spacy
print('Spacy precision score is',tp_spacy / (tp_spacy + fp_spacy))
print('Spacy recall score is',tp_spacy / (tp_spacy + fn_spacy))

NLTK precision score is 0.062557497700092
NLTK recall score is 0.3938223938223938
Spacy precision score is 0.09941520467836257
Spacy recall score is 0.22972972972972974
